**Copyright 2021 Antoine SIMOULIN.**

<i>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

Icons made by <a href="https://www.flaticon.com/authors/freepik" title="Freepik">Freepik</a>, <a href="https://www.flaticon.com/authors/pixel-perfect" title="Pixel perfect">Pixel perfect</a>, <a href="https://www.flaticon.com/authors/becris" title="Becris">Becris</a>, <a href="https://www.flaticon.com/authors/smashicons" title="Smashicons">Smashicons</a>, <a href="https://www.flaticon.com/authors/srip" title="srip">srip</a>, <a href="https://www.flaticon.com/authors/adib-sulthon" title="Adib">Adib</a>, <a href="https://www.flaticon.com/authors/flat-icons" title="Flat Icons">Flat Icons</a> and <a href="https://www.flaticon.com/authors/dinosoftlabs" title="Pixel perfect">DinosoftLabs</a> from <a href="https://www.flaticon.com/" title="Flaticon"> www.flaticon.com</a></i>

# TP2 - Topic Modeling

<img src="./figures/figure2.png" width="1000">

Le <i>Topic Modeling</i> est une approche statistique qui permet de faire émerger des topics abstraits d'un corpus de documents. 
Cette approche permet également d'analyser la structure du corpus de documents en regroupant ceux qui présentent des topics similaires puis en analysant ces groupes, ou en analysant les caractéristiques des topics identifiés.

La plupart des modèles de <i>Topic Modeling</i> s'appuient sur des hypothèses de modélisations similaires:
* Chaque document est modélisé comme une distribution sur les _topics_ ;
* Chaque _topic_ est modélisé comme une distribution sur les mots du vocabulaire.

On a illustré cette modélisation ci-dessous. Ainsi chaque document est représenté par une distribution sur une variable latente (on dit aussi cachée), les _topics_. Ces derniers ne sont pas "observés" : en pratique chaque document est décrit par une distribution sur les mots du vocabulaire. **L'objectif des modèles de _topics_ est donc de caractériser la forme de cette variable latente.** Nous allons voir plusieurs méthodes et modèles proposant cette caractérisation.

Ci-dessous, on a illustré l'intuition derrière cette modélisation. Chaque document va contenir plusieurs _topics_, par exemple, les transports et les vacances. On retrouvera donc des mots caractéristiques de ces topic: "avion", "plage", "congés" ... Des documents qui abordent des _topics_ proches contiendront donc un vocabulaire proche. Ainsi chaque _topic_ pourra être caractérisé par des mots saillants qui lui sont spécifiques.



<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/lda-idee.png?raw=true" width="1000">


In [ ]:
%%capture

# ⚠️ Execute only if running in Colab
if 'google.colab' in str(get_ipython()):
  IN_COLAB = True
else:
  IN_COLAB = False

if IN_COLAB:
  !pip install -q scikit-learn==0.23.2 nltk==3.5 unidecode pysrt
  !pip install --no-deps pyLDAvis==3.3.1
  !pip install --no-deps funcy==1.16

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import numpy as np
import os
from pyLDAvis import sklearn as sklearn_lda
import pickle
import pyLDAvis
import pysrt
import re
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS
from tqdm.auto import tqdm
import unidecode
import urllib.request

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

In [ ]:
# import extrenal modules

repo_url = 'https://raw.githubusercontent.com/AntoineSimoulin/m2-data-sciences/master/'
for season in range(1, 9):
  dir = './data/S{:02d}'.format(season)
  if not os.path.exists(dir):
    os.makedirs(dir)
    for episode in range(1, 11):
      try:
        _ = urllib.request.urlretrieve(
            repo_url + 'TP2%20-%20Text%20Mining/sous-titres-got/S{:02d}/E{:02d}.srt'.format(season, episode), 
            './data/S{:02d}/E{:02d}.srt'.format(season, episode))
      except:
        pass

## Latent Semantic Analysis (LSA)

Le modèle Latent Semantic Analysis (LSA) ([Landauer & Dumais, 1997](#landauer-dumais-1997)) cherche à décomposer la matrice de décomposition des documents selon le vocabulaire en deux matrices : une matrice de décomposition des documents selon les topics et une matrice de distribution des topics selon les mots du vocabulaires.

On commencer donc par représenter les documents selon une distribution sur le vocabulaire. Pour cela on utilise le Tf-Idf qui permet de représenter chaque document du corpus comme une distribution sur le vocabulaire, en pratique, un vecteur de la taille du vocabulaire. On peut donc représenter le corpus comme une matrice de taille $(M, V)$ avec $M$ le nombre de documents dans le corpus et $V$ la taille du vocabulaire. Cette représentation est illustrée ci-dessous. 

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/bow.png?raw=true" width="500">

On va ensuite décomposer la matrice en utilisant **décomposition en valeurs singulières** (en anglais Singular Value Decomposition, [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition)). On peut interpréter la SVD comme la généralisation de la diagonalisation d'une matrice normale a des matrices arbitraires. Ainsi une matrice $A$ de taille $n \times m$ peut être factorisée sous la forme $A = U \Sigma V^T$, avec $U$ et $V$ des matrices orthogonales de tailles respectives $m \times m$ et $n \times n$ et $\Sigma$ une martice rectangulaire diagonale de taille $m \times n$. 

En pratique, il est peu commun de décomposer complétement la matrice, on utilise plutôt la <a href="https://en.wikipedia.org/wiki/Singular_value_decomposition#Truncated_SVD"><i>Trucated Singular Value Decomposition</i></a> qui permet de ne calculer que les $t$ premières valeurs singulières. Dans ce cas, on ne considère que les $t$ premières colonnes de la matrice $U$ et les $t$ premières lignes de la matrice $V$. On a ainsi :

$$A_t = U_t \Sigma_t V_t^T$$

Avec $U_t$ de taille $m \times t$ et $V_t$ de taille $n \times t$. Cette décomposition est illustrée ci-dessous.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/svd-formule.png?raw=true" width="500">

On illustre ci-dessous l'application de la décomposition à notre matrice Tf-Idf. La matrice $U_t$ apparait comme la matrice <i>document-topic</i> qui définit chaque document comme une distribution de topic. La matrice $V_t$ apparait comme la matrice <i>terme-topic</i> qui définit chaque topic comme une distribution sur le vocabulaire.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/svd-illustration.png?raw=true" width="500">

On peut aussi interpréter le <i>Topic Modeling</i> comme une approche de réduction de dimension. En effet, la matrice Tf-Idf a plusieurs défauts : Elle est de grande dimension (la taille du vocabulaire), elle est _sparse_ _i.e._ beaucoup d'entrées sont à zéro, elle est très bruitée et les information sont redondantes selon plusieurs dimensions. La décomposition permet ainsi de la factoriser. Les deux matrices résultantes permetent d'utiliser la similarité cosinus pour comparer simplement des doccuments ou des mots.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
documents = ["doc1.txt", "doc2.txt", "doc3.txt"] 
  
# raw documents to tf-idf matrix: 
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)
# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=100,         // num dimensions
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

## Probabilistic Latent Semantic Analysis (pLSA)

La LSA est une méthode très efficace. Néanmoins en pratique, les topics résultants sont parfois difficiles à interpréter. 
La méthode nécessite un corpus important pour obtenir des résultats pertinents.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/plda_principe.png?raw=true" width="500">

La methode de Probabilistic Latent Semantic Analysis (pLSA) remplace ainsi la SVD par une approche probabiliste.
Il s'agit d'un modèle **génératif**, qui permet de générer les documents que l'on observe. 
En pratique il permet de générer la matrice Bag-of-words qui représente le corpus. Le modèle ne tient donc pas compte de l'ordre des mots.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/plda.png?raw=true" width="500">

Le fonctionnement du modèle est détaillé selon la représentation graphique suivante :
* Etant donné un document $d$, un topic $z$ est présent dans le document avec une probabilité $P(z|d)$. 
* Etant donné un topic $z$, un mot est généré selon la probabilité conditionnelle $P(w|z)$.

La probabilité jointe d'observer un mot dans un document s'exprime donc :

$$P(D,W)=P(D)\sum_ZP(Z|D)P(W|Z)$$

Ici, $P(D)$, $P(Z|D)$ et $P(W|Z)$ sont des paramètres du modèle. $P(D)$ peut être  calculé directement à partir du corpus.
$P(Z|D)$ et $P(W|Z)$ sont modélisés par des distributions multinomiales qui peuvent être entrainés par la méthode [EM](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm).

En pratique, on apprend donc les paramètres de notre modèles qui permetent d'expliquer qu mieux le corpus observé comme illustré ci-dessous.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/plsa_inference.png?raw=true" width="500">


## Latent Dirichlet Allocation (LDA)

La Latent Dirichlet Allocation (LDA) <span class="badge badge-secondary">(Blei et al., 2001)</span> est une méthode de <b>topic modelling</b>. C'est l'une des méthodes les plus utilisées dans ce domaine. La LDA prend en input une collection de documents et cherche à identifier les topics ou thèmes spécifiques dans l'ensemble du corpus.

<span class="badge badge-secondary">(Blei et al., 2001)</span> David M. Blei, Andrew Y. Ng, Michael I. Jordan: Latent Dirichlet Allocation. NIPS 2001: 601-608

Par exemple, on peut considérer le corpus suivant :

In [1]:
corpus = [
    "Nous avons pris l'avion pour aller en vacances.",
    "J'ai fait de la plongée en vacances."
    "Nous avons joué au foot hier matin.",
    "J'ai pris le taxi.",
]

In [2]:
n_documents = len(corpus)

L'objectif est d'identifier les thèmes ou <i>topics</i> qui correspondent à chaque phrase. Ici, on pourrait considérer des thèmes commes les <i>vacances</i>, le <i>sport</i> ou encore les <i>moyens de transports</i>.

La LDA est un modèle <b>génératif</b> qui cherche à expliquer une observation (ici le corpus) en s'appuyant sur des variables latentes (les topics). En plus de l'exploration de thèmes, ce type de modèle peut être utilisé pour des tâches non supervisées, en particulier le clustering.

La LDA décrit chaque document comme une distribution de **topics**, eux mêmes caractérisés par une **distribution de mots**. En pratique, on va introduire une **variable latente** et exprimer chaque document comme une distribution de cette variable. Cette variable sera elle même décrite comme une distribution sur le vocabulaire comme décrit ci-dessous. Ainsi chaque topic n'est pas décrit explicitement. Il doit être interprété en fonction de sa distribution sur les mots du vocabulaire.

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/lda-idee.png?raw=true" width="1000">



In [4]:
vectorizer = CountVectorizer()

vectorizer.fit(corpus)

w2idx = {w: i for (i, w) in enumerate(vectorizer.get_feature_names())}
idx2w = {i: w for (i, w) in enumerate(vectorizer.get_feature_names())}

print(len(w2idx))

20


In [5]:
tokenizer = vectorizer.build_tokenizer()

In [6]:
corpus = [[t.lower() for t in tokenizer(d)] for d in corpus]
for document in corpus:
    print(document)

['nous', 'avons', 'pris', 'avion', 'pour', 'aller', 'en', 'vacances']
['ai', 'fait', 'de', 'la', 'plongée', 'en', 'vacances', 'nous', 'avons', 'joué', 'au', 'foot', 'hier', 'matin']
['ai', 'pris', 'le', 'taxi']


In [7]:
vocab_size = len(vectorizer.get_feature_names())
print(vocab_size)

20


In [8]:
print("matrice d'occurence :")

n = np.zeros((len(corpus), vocab_size), dtype=int)
for (document_idx, document) in enumerate(corpus):
    words_idx, words_freq = np.unique(document, return_counts=True)
    for (w, f) in zip(words_idx, words_freq):
        n[document_idx][w2idx[w]] = f
print(n)

matrice d'occurence :
[[0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1]
 [1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0]]


## 1. Modélisation

La séance précédente, on a vu que l'on pouvait représenter les documents comme une distribution sur les mots. Les paramètres de cette distribution peuvent être calculés simplement en utilisant la fréquence des mots dans le document (le modèle Bag-Of-Word) ou alors en pondérant les mots en fonction de leur fréquence dans l'ensemble du corpus (modèle TF-IDF).

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/bow.png?raw=true" width="500">

### Les modèles génératifs

La LDA est un modèle de probabilité génératif. Les modèles génératifs sont générallement développés suivant les 3 étapes suivantes :

1. **Observations** Les données sont considérées comme des observations générées par le modèle. La notion de données inclue les variables latentes. Ces dernières représentent la structure thématique du corpus.
2. **Apprentissage** On met à jour les variables du modèle en utilisant l'inférence à postériori
3. **Inférence** On peut utiliser le modèle pour situer de nouvelles de données dans la structure de topics apprise.

L'originalité de la LDA réside dans sa modélisation de chaque document comme une distribution de topics. La plupart des modèles considérant que tous les mots d'un document sont issus du même topic. Ainsi chaque document peut être rattaché à plusieurs topics. Dans notre corpus exemple, la LDA pourrait générer une distribution du type :

* Document 1 : 50% Topic "Transports" + 50% Topic "Voyages"
* Document 2 : 50% Topic "Sports" + 50% Topic "Voyages"
* Document 3 : 100% Topic "Sports"
* Document 4 : 100% Topic "Transports"

<img src="https://github.com/AntoineSimoulin/m2-data-sciences/blob/master/TP2%20-%20Text%20Mining/figures/lda2.png?raw=true" width="1000">

### Notations

Les modèles graphiques représentent les variables aléatoies comme des noeuds. Les arcs entre les noeuds indiquent les variables potentiellement dépendantes. Les variables observées sont grisées. Dans la figure ci-dessous, les noeuds $ X_{1,...,N}$ sont observés alors que le noeud $Y$ est une variable latente. Dans cet exemple, les variables observées dépendent de cette variable latente. Les rectangles synthétisent la réplication de plusieurs structures. Un rectangle résume donc plusieurs variables $X_n$ avec $n \in N$.

La structure du graph définie les dépendances conditionnelles entre l'ensemble des variables. Par exemple dans le graph ci-dessous, on a $p(Y,X_{1},...,X_{N})=p(Y)\prod _{n=1}^{N}p(X_{n}|Y)$.

<img src="./figures/graphical_model.png" width="500">

### La distribution de Dirichlet

La distribution de Dirichlet est généralement notée $Dir(\alpha)$. Il s'agit d'une famille de lois de probabilités continues pour des variables aléatoires multinomiales. Cette loi (ou encore distribution) est paramétrée par le vecteur ${\bf \alpha}$ de nombres réels positifs. La taille du vecteur ${\bf \alpha}$ indique la dimension de la distribution. Ce type de distribution est souvent utilisée comme distribution à priori dans les modèles Bayésiens. Sans rentrer dans les détails, voici quelques caraactéristiques de la distribution de Dirichlet :

* La distribution est définie sur un simplex de vecteurs positifs dont la somme est égale à 1 
* Sa densité est caractérisée par : $P(\theta |{\overrightarrow {\alpha }})={\frac {\Gamma (\Sigma _{i}\alpha _{i})f}{\Pi _{i}\Gamma (\alpha _{i})}}\Pi _{i}\theta _{i}^{\alpha _{i}-1}$
* En pratique, si toutes les dimensions de ${\bf \alpha}$ ont des valeurs similaires, la distribution est plus étalée. Elle devient plus concentrée pour des valeurs plus importantes de ${\bf \alpha}$.

La distribution est illustrée ci-dessous pour des valeurs ${\bf \alpha}$ qui varient entre (6, 2, 2), (3, 7, 5), (6, 2, 6), (2, 3, 4).

<img src="./figures/dirichlet.png" width="500">

On utilise les notations suivantes :

* Les mots sont l'unité de base des données. Ils sont définis comme un élément d'un vocabulaire indexé par ${1,...,V}$. On représent chaque mot en utilisant des vecteurs one-hot dont toutes les composantes sont 0, sauf pour la composante qui correspond à l'index du mot, qui vaut 1.
* Un document est une séquence de $N$ mots telle que $W=(w_{1},w_{2},...,w_{N})$ avec $w_{n}$ le $n$th mot de la séquence.
* Un corpus est un ensemble de $M$ documents tels que $D=\lbrace W_{1},W_{2},...,W_{M}\rbrace$.

### Le principe de la LDA

La LDA est un modèle génératif. L'idée de base est que chaque document est représenté comme une distribution sur $k$ topics latents. Chaque topic est caractérisé par une distribution sur les mots. On appelle $\beta$ la matrice de dimension $k*V$ qui représente la distribution des mots sur les $k$ topics. On a ainsi $\beta _{ij}=p(w_{j}=1|z_{i}=1)$.

La LDA suppose le procéssus génératif suivant pour chaque document $W$ dans le corpus $D$.


> 1. On choisit $\theta \sim Dir(\alpha )$
> 2. Pour chaque document dans le corpus:
>     * Pour chacun des $N$ mots $w_{n}$ dans le document :
>        * on génère un topic $z_{n}\sim Multinomial(\theta )$
>        * on génère un mot $w_{n}$ $p(w_{n}|z_{n},B)$ selon une loi multinimoale conditionnée par le topic $z_{n}$.


Une variable aléatoire suivant une loi de dirichlet de dimension $k$ peut prendre des valeurs dans le $k-1$-simplex. Cet espace désigne les vecteurs $\theta$ de dimension $k$ tels que $\theta _{i}\geq 0$ et $\sum _{i=1}^{k}\theta _{i}=1$

Etant donné $\alpha$ et $\beta$, la probabilité jointe de $\theta$, un ensemble de $K$ topics $Z$ et un ensemble de $N$ mots est donnée par : $$p(\theta ,Z,W|\alpha ,\beta )=p(\theta |\alpha )\prod _{n=1}^{N}p(z_{n}|\theta )p(w_{n}|z_{n},\beta ),$$

Avec $p(z_{n}|\theta )$ qui représente $\theta _{i}$ pour chaque $i$ tel que $z_{i}^{n}=1$.

On peut obtenir la distribution marginale du document en itérant sur les $\theta$ et en sommant sur les $z$:

$$p(W|\alpha ,\beta )=\int p(\theta |\alpha ){\big (}\prod _{n=1}^{N}\Sigma _{z_{n}}p(z_{n}|\theta )p(w_{n}|z_{n},\beta {\big )}d\theta$$

Finalement, en prenant le produit des probabilitées marginales sur un chaque document, on obtient la probabilité du corpus :


$$p(D|\alpha ,\beta )=\prod _{d=1}^{M}\int p(\theta _{d}|\alpha ){\big (}\prod _{n=1}^{N_{d}}\Sigma _{z_{dn}}p(z_{dn}|\theta _{d})p(w_{dn}|z_{dn},\beta ){\big )}d\theta _{d}$$

La représentation garphique du modèle est proposée ci-dessous.

<img src="./figures/lda_graph.png" width="700">

Avec les notations suivantes :
* k — Le nombre de topics qui caractérisent un document
* V — La taille du vocabulaire
* M — Le nombre de documents
* N — Le nombre de mots dans chaque document
* w — Un mot dans un document représenté par un vecteur one-hot de taille V
* **w** — Représentation d'un document, i.e. une collection de N vecteurs w de mots
* D — Le corpus, i.e. une collection de M documents
* z — Un topic parmi k. Un topic correspond à une distribution de mots. Par exemple, voyage = (0.3 avion, 0.4 plongée, 0.2 vacances)

## 2. Apprendre les paramètres du modèle

Ce modèle permet d'expliquer comment le corpus a été généré. Néanmoins en pratique on n'observe pas toutes les variables du modèle mais seulement la distribution des mots. En pratique on va "inverser" le modèle pour estimer les paramètres. Ce processus est appelé inférence à posteriori.

On dispose d'un corpus de documents. On a fixé un nombre $k$ de topics. Pour apprendre la représentation des topics et des documents, il existe deux méthodes principales :

* Le sampling de Gibbs
* L'inférence variationelle

Nous allons détailler le procéssus du sampling de Gibbs sur notre jeu de données exemple puis utiliser la librarie `sklearn` qui repose sur l'inférence variationelle pour expérimenter sur un jeu de données plus conséquent.

In [10]:
n_topics = 3

### Le Sampling de Gibbs

Pour fixer les paramètres des matrices, on cherche à maximiser la vraisemblance de nos données selon ce modèle. On utilise pour cela l'algorthime de **sampling de Gibbs**. Le sampling de Gibbs est un algorithme qui permet de sélectionner des distributions conditionelles dont la distribution des états converge vers la vraie distribution à terme. En pratique, on va mettre à jour itérativement les matrices $\Theta$ et $\beta$ pour maximiser la vraisemblance de nos données. Les itérations s'effectuent mot par mot en ajustant le topic assigné à chaque mot.  On fait l'hypothèse que l'on ne connait pas le topic assigné à chaque mot mais qu'on connait la correpondance de topic pour tous les autres mots dans le texte et on cherche à inférer le topic à assigner pour ce mot.


D'un point de vue mathématique, on cherche la probabilité conditionnelle pour chaque mot d'être assigné à un topic, étant donné l'attribution des autres topics. On peut montrer que peut s'écrire :

$$P(z_{i,d}=k|z_{:,d},w,\alpha,\beta) \propto \frac{n_{d,k}+\beta_k}{\sum_i^Kn_{d,i}+\beta_i}v_{k,w_{d,n}}+\alpha_{w_{d,n}}$$

Avec :
* $n_{d,k}$ : le nombre de fois ou le document $d$ utilise le topic $k$
* $v_{k,w}$ : le nombre de fois ou le topic $k$ utilise le mot $w$
* $\alpha_k$ : le paramètre de dirichlet pour la distribution de topics par documents
* $\lambda_w$ : Le paramètre de dirichlet pour la distribution des mots par topic

Il y a deux parties dans cette équation. D'abord on évalue la proportion de chaque topic dans un document. Puis on réparti l'attention des topics pour chaque mot. Les paramètres de dirichlet permette de régulariser quand $n_{d,k}$ ou $v_{k,w}$ sont trop proches de 0 et qu'il y a peu de chance qu'un mot choisise un topic.

In [11]:
#  1. Pour chaque document et chaque mot, assigner un topic initial au hasard

words_topic = [[np.random.randint(n_topics) for _ in range(len(d))] for d in corpus]

print("assignation de topic aléatoire pour le premier document")
for (w, t) in zip(corpus[0], words_topic[0]):
    print("{:>10} -> TOPIC {:1}".format(w, t))
    
print("matrices d'assignation des topics :")
words_topic

assignation de topic aléatoire pour le premier document
      nous -> TOPIC 1
     avons -> TOPIC 1
      pris -> TOPIC 2
     avion -> TOPIC 1
      pour -> TOPIC 2
     aller -> TOPIC 2
        en -> TOPIC 0
  vacances -> TOPIC 1
matrices d'assignation des topics :


[[1, 1, 2, 1, 2, 2, 0, 1],
 [1, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1],
 [2, 2, 1, 2]]

* $n_{d,k}$ : le nombre de fois ou le document $d$ utilise le topic $k$
* $v_{k,w}$ : le nombre de fois ou le topic $k$ utilise le mot $w$

In [12]:
# 2. Etant donné cette distribution de topic, calculer la distribution des documents
#    en fonction des topics et des topics en fonction des mots

document_topic_counts = np.zeros((len(corpus), n_topics), dtype=int)
for (doc_idx, topics) in enumerate(words_topic):
    topics, topics_freq = np.unique(topics, return_counts=True)
    for (t, f) in zip(topics, topics_freq):
        document_topic_counts[doc_idx][t] = f
print("Le nombre de fois ou le document  𝑑  utilise le topic  𝑘 :")
print(document_topic_counts, '\n')

topic_word_counts = np.zeros((n_topics, vocab_size + 1), dtype=int)
for (topics, document) in zip(words_topic, corpus):
    for (t, w_idx) in zip(topics, document):
        topic_word_counts[t][w2idx[w_idx]] += 1
print("Le nombre de fois ou le topic  𝑘  utilise le mot  𝑤 :")
print(topic_word_counts)

Le nombre de fois ou le document  𝑑  utilise le topic  𝑘 :
[[1 4 3]
 [1 7 6]
 [0 1 3]] 

Le nombre de fois ou le topic  𝑘  utilise le mot  𝑤 :
[[0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 2 0 0 0 1 0 1 0 1 1 2 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 1 2 1 1 0]]


In [13]:
# nombre de mots pour chaque topic:
np.sum(topic_word_counts, 1)

array([ 2, 12, 12])

In [14]:
# 3. Pour affiner les distributions définies précédemments on va itérer sur l'ensemble des documents
#    3.1 Pour chaque mot de chaque document :
#        On calcule la probabilité conditionnelle du mot d'être assigné à un topic en utilisant la formule ci-dessus
#        suivant cette probabilité, on réassigne le topic du mot considéré.
#        En fait, on considère ici que seul la probabilité du mot considéré est à ajuster et que toutes les autres sont correctes.
#        Cette version de l'algorithme est assez simplifié pour mieux comprendre le procéssus.

eta = 0.01
alpha = 50.


for (doc_idx, doc) in enumerate(corpus):
    for (w_idx, w) in enumerate(doc):
        
        
        current_topic = words_topic[doc_idx][w_idx]
        
        # Mise à jour des matrices de fréquences
        document_topic_counts[doc_idx, current_topic] -= 1
        topic_word_counts[current_topic, w2idx[w]] -= 1
        
        # Change le topic
        topic_distribution = (topic_word_counts[:, w2idx[w]] + eta) * \
            (document_topic_counts[doc_idx, :] + alpha) / \
            (np.sum(topic_word_counts, 1) + 1e-12)

        new_topic = np.random.multinomial(1, topic_distribution / topic_distribution.sum()).argmax()
        
        # Mise à jour des matrices de fréquences
        document_topic_counts[doc_idx][new_topic] += 1
        topic_word_counts[new_topic, w2idx[w]] += 1
        words_topic[doc_idx][w_idx] = new_topic

In [15]:
document_topic_counts

array([[2, 2, 4],
       [4, 6, 4],
       [0, 0, 4]])

In [16]:
topic_word_counts

array([[0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 1, 2, 0]])

In [17]:
# 4. On va répéter cette transformation un certain nombre de fois afin de faire converger la distribution
#    des topics

n_iters = 10

for _ in range(n_iters):
    for (doc_idx, doc) in enumerate(corpus):
        for (w_idx, w) in enumerate(doc):


            current_topic = words_topic[doc_idx][w_idx]

            # Mise à jour des matrices de fréquences
            document_topic_counts[doc_idx, current_topic] -= 1
            topic_word_counts[current_topic, w2idx[w]] -= 1

            # Change le topic
            topic_distribution = (topic_word_counts[:, w2idx[w]] + eta) * \
                (document_topic_counts[doc_idx, :] + alpha) / \
                (np.sum(topic_word_counts, 1) + 1e-12)

            new_topic = np.random.multinomial(1, topic_distribution / topic_distribution.sum()).argmax()

            # Mise à jour des matrices de fréquences
            document_topic_counts[doc_idx][new_topic] += 1
            topic_word_counts[new_topic, w2idx[w]] += 1
            words_topic[doc_idx][w_idx] = new_topic

In [18]:
document_topic_counts

array([[3, 4, 1],
       [7, 2, 5],
       [1, 1, 2]])

Finalement on peut évaluer les matrices $\Theta$ et $\beta$ selon les formules suivantes :

$$\Theta_{k,t} = \frac{n_{d,k}+\eta_k}{\sum_i^Kn_{d,i}+\eta_k}$$

$$\beta_{m,k} = \frac{v_{k}+\alpha_k}{\sum_i^Kn_{d,i}+\alpha_k}$$


In [19]:
beta = (topic_word_counts + alpha) / (np.sum(topic_word_counts, 0) + alpha)

In [20]:
n_top_words = 3

for topic_idx in range(n_topics):
    message = "Topic #%d: " % topic_idx
    message += " ".join([idx2w[i] for i in beta[topic_idx].argsort()[:-n_top_words - 1:-1][1:]])
    print(message)

Topic #0: matin aller
Topic #1: pris pour
Topic #2: le au


## 3. Utilisation des librairies

On va chercher à analyser les thèmes de la Série Game Of Thrones. On utilise pour ça les sous-titres de l'ensemble des saisons qui ont été récupérés sur le site https://www.sous-titres.eu/series/game_of_thrones.html.

In [22]:
def create_subtitle_file_dict(subtitles_dir):
    "Retourne les chemins vers les fichiers de sous titres"
    subtitles_file_path = {}
    for path, subdirs, files in os.walk(subtitles_dir):
        for name in files:
            episode_name = '_'.join([os.path.basename(path), name.split('.')[0]])
            subtitles_file_path[episode_name] = os.path.join(path, name)
    return subtitles_file_path

def parse_srt_file(srt_file, encoding='iso-8859-1'):
    "Lit un ficher de sous titres au format rst"
    subs = pysrt.open(srt_file, encoding=encoding)
    text = ' '.join([' '.join(sub.text.split('\n')) for sub in subs])
    return text

def create_corpus(subtitles_file_path):
    "Créer un corpus à partir de tous les fichiers rst dans un dossier"
    corpus = []
    for k, v in subtitles_file_path.items():
        if v.endswith('srt'):
            corpus.append(parse_srt_file(v))
    return corpus

In [23]:
subtitles_file_path = create_subtitle_file_dict('./data/')

In [24]:
episode_1_txt = parse_srt_file(subtitles_file_path['S01_E01'])

In [25]:
print(episode_1_txt[:100])

Doucement. Que croyais-tu ? Ce sont des sauvages. L'un vole une chèvre à l'autre, et ils finissent p


In [26]:
corpus = create_corpus(subtitles_file_path)

In [27]:
len(corpus)

73

In [28]:
corpus[0][:100]

"Attention ! Au feu ! Au feu ! Une vingtaine d'hommes, voire moins. Ils se sont faufilés dans le camp"

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Nettoyer le corpus pour enlever les accents, mettre le texte en minuscule, enlever la ponctuation et les doubles espaces. Eventuellement pour le stemming </p>
</div>    

In [29]:
# %load solutions/cleaning.py
stemmer = FrenchStemmer()

def clean_corpus(corpus):
    for i in range(len(corpus)):
        corpus[i] = unidecode.unidecode(corpus[i])
        corpus[i] = re.sub(r'[^\w\s]', ' ', corpus[i])
        corpus[i] = corpus[i].lower()
        corpus[i] = re.sub(r'\s{2,}', ' ', corpus[i])
        # corpus[i] = ' '.join([stemmer.stem(x) for x in corpus[i].split()])
    return corpus

In [30]:
clean_corpus = clean_corpus(corpus)

In [31]:
clean_corpus[0][:100]

'attention au feu au feu une vingtaine d hommes voire moins ils se sont faufiles dans le camp ils ont'

In [32]:
clean_corpus_split = []
for episode in clean_corpus:
    episode_words = episode.split()
    i = 0
    while i < len(episode_words):
        clean_corpus_split.append(' '.join(episode_words[i:i+400]))
        i+=400

In [33]:
len(clean_corpus_split)

726

In [34]:
def tokenize_corpus(corpus):
    tokens = []
    for sentence in corpus.split('\n'):
        tokens.append(nltk.word_tokenize(sentence))
    return tokens

In [35]:
sentence_length = [len(x.split()) for x in clean_corpus_split]

In [36]:
np.mean(sentence_length), np.std(sentence_length)

(378.10743801652893, 75.55046495285958)

<div class="alert alert-info" role="alert">
    <p><b>Exercice :</b> Vectorizer le corpus en utilisant la méthode Bag-Of-Words.</p>
</div>    

In [37]:
# %load solutions/vectorize.py

# Initialise the count vectorizer
count_vectorizer = CountVectorizer(max_features=2000,
                                   stop_words=STOP_WORDS,
                                   max_df=0.9,
                                   min_df=20)

count_data = count_vectorizer.fit_transform(clean_corpus_split)

/Users/antoine/opt/anaconda3/envs/nlp-101/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [38]:
len(clean_corpus_split)

726

In [39]:
# Tweak the two parameters below
number_topics = 15
number_words = 10

# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)

LatentDirichletAllocation(n_components=15, n_jobs=-1)

In [40]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [41]:
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
avez bolton pere stark qu nom lord nord ramsay famille

Topic #1:
stark lord pere roi nord lannister robb winterfell maison fils

Topic #2:
qu ca mort nuit garde mur hommes veux theon sais

Topic #3:
qu pere ca mort sais etes mere jamais fils avez

Topic #4:
grand reine loras ca tyrell margaery foi dieux qu oui

Topic #5:
qu ca mur faire avez tue jamais hommes garde roi

Topic #6:
khaleesi khal qu drogo veux dothraki sang allez faire etes

Topic #7:
qu roi reine nord avez hommes ici guerre armee sais

Topic #8:
verite jeu petite trone fortune navires prince braavos femme avez

Topic #9:
qu roi fils avez homme lannister hommes etes frere ser

Topic #10:
avez reine meereen etes maitres pere ici westeros daenerys esclaves

Topic #11:
capitaine majeste avez etes faire paix faut confiance reine renly

Topic #12:
dubbing brothers titrage adaptation blandine menard clotilde maville qu ca

Topic #13:
ca pitie qu frere maitre hommes nom homme fils dois

Topic #1

## 4. Visualisation

In [43]:
%%time


LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(number_topics))
LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer, mds='mmds')

CPU times: user 591 ms, sys: 115 ms, total: 706 ms
Wall time: 2.06 s


In [44]:
with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(number_topics) +'.html')

In [45]:
pyLDAvis.display(LDAvis_prepared)

<span class="badge badge-secondary">(Sievert et al., 2014)</span> Sievert, Carson, and Kenneth Shirley. "LDAvis: A method for visualizing and interpreting topics." Proceedings of the workshop on interactive language learning, visualization, and interfaces. 2014.

<span class="badge badge-secondary">(Chuang et al., 2012)</span> Chuang, Jason, Christopher D. Manning, and Jeffrey Heer. "Termite: Visualization techniques for assessing textual topic models." Proceedings of the international working conference on advanced visual interfaces. 2012.

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Faire varier le paramètre Lambda et justifier de son impact.</p>
</div>
<hr>

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Faire varier le préprocessing,en particulier la stemmatization. Analyser l'impact sur l'analyse des clusters.</p>
</div>
<hr>

<hr>
<div class="alert alert-info" role="alert">
    <p><b>📝 Exercice :</b> Etudier l'impact des Stop Words sur les topics.</p>
</div>
<hr>

## References

> <div id="landauer-dumais-1997">Landauer, Thomas K. et al. “<a href=http://lsa.colorado.edu/papers/dp1.LSAintro.pdf>An introduction to latent semantic analysis.</a>” Discourse Processes 25 (1998): 259-284.</div>

Sources :
* https://medium.com/analytics-vidhya/topic-modeling-using-lda-and-gibbs-sampling-explained-49d49b3d1045
* https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158
* https://wiki.ubc.ca/Course:CPSC522/Latent_Dirichlet_Allocation
* http://www.arbylon.net/publications/text-est2.pdf

